# RAG ror PDF written purely in Python

First install a library to read pdf-files

In [2]:
# !pip install pymupdf

# Build the vector store

First we build the vector store based on our main document. Here we downloaded some paper.

In [3]:
the_document_path = "./data/2205.13147v4.pdf" 

In [4]:
from py4ragTools.database import Database

In [5]:
# embedding_model = 'all-minilm'  # 384
embedding_model='nomic-embed-text'  # 768
db = Database(embedding_model=embedding_model)

constructor here


In [6]:
from py4ragTools.text_tools import TextHelper, CharacterTextSplitter
th = TextHelper()

# load the document
doc = th.load(the_document_path)

# split the text into chunks
splitter = CharacterTextSplitter(chunk_size=800,chunk_overlap=200)
chunks = splitter.split(doc)
len(chunks)

212

In [7]:
# now we build the 'vector store' converting each chunk into a vector
count = 0
for chunk in chunks:
    # Bei Texten der Länge 0 bleibt ollama stehen
    # Sehr kurze Texte sind unbrauchbar für's anschliessende Generieren
    if len(chunk) > 10:
        # print(count)
        # print(entry)
        db.add(chunk)
        count += 1
print(f'{count} entries in db')  

212 entries in db


# Retrieval

We have built our vector store let us retrieve a context for a user-query.

Set up a query regarding today's topics on the newspaper's front page.

In [8]:
# okn: Task 2/3 trying some prompt pattern and test some specific user prompts
# okn: observ: sentance 'it is often the case...' available in context, but not in response


# okn: text passage 'constaint not known in advance' im context, aber nicht im nicht im response erwähnt
USER_PROMPT_0 = """
Are there computational and statistical constraints?
"""

# okn: observ: context has complete sub points, response only returns first
USER_PROMPT_1 = """ \
What does the Checklist say, if you are using existing assets?
"""

# audience prompt pattern
# okn: observ: response sounds well, though hard to say if it fully makes sense
USER_PROMPT_2 = """ \
What is Matryoshka Representation Learning? Explain to me like I’m 11 years old.
"""

# persona  pattern
# okn: observ: response sounds well, though hard to say if it fully makes sense
USER_PROMPT_3 = """ \
What is Matryoshka Representation Learning? Act as a oriental story teller.
"""

# response template pattern
# okn: observ: works well
USER_PROMPT_4 = """ \
What is Matryoshka Representation Learning?
Format your answer as follows:
#Question: [The question being asked by User]
## Summary 
[a one paragraph summary of the answer]
## Details 
[a detailed answer]
"""

# question refinement pattern
# okn: works as long as there is some context returned by the vector store
USER_PROMPT_5 = """ \
Is Matryoshka Representation usefull at all?
If question is not clearly formulated, suggest a better version of the question and ask user if he would like to use it instead.
"""

user_query = USER_PROMPT_0


In [9]:
# okn: observ: have adjustet the parameter until it returned the document text in Task 3
context = db.query_database(user_query, 0.5 )
len(context)


56

In [10]:
context

[(0.5845468924444428,
  ' Mu, S. Kadavath, F. Wang, E. Dorundo, R. Desai, T. Zhu,\nS. Parajuli, M. Guo, et al. The many faces of robustness: A critical analysis of out-of-\ndistribution generalization. In Proceedings of the IEEE/CVF International Conference on\nComputer Vision, pages 8340–8349, 2021.\n12\n[35] D. Hendrycks, K. Zhao, S. Basart, J. Steinhardt, and D. Song. Natural adversarial examples.\nIn Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition,\npages 15262–15271, 2021.\n[36] S. Hooker, A. Courville, G. Clark, Y. Dauphin, and A. Frome. What do compressed deep\nneural networks forget? arXiv preprint arXiv:1911.05248, 2019.\n[37] S. Hooker, N. Moorosi, G. Clark, S. Bengio, and E. Denton. Characterising bias in compressed\nmodels. arXiv preprint arXiv:2010.03058, 2020.\n[38] H. Hotelling'),
 (0.5442247528071253,
  'Lab/robustness.\n[25] A. Gholami, S. Kim, Z. Dong, Z. Yao, M. W. Mahoney, and K. Keutzer. A survey of\nquantization methods for effici

okn: doument chunk found "computational and statistical constraints" for PROMPT_0
- generally the context looks of low quality  (lots of numbers figures without semantics, broken sentences etc. )

# Augmentation
Now that we have built our "vector database" and retrieved our context, we do the inference part:

First, we augment the prompt using the context we retrieved.

In [11]:
from py4ragTools.prompt_tools import PromptTools

pt = PromptTools()

In [12]:
# okn: test some other system templates

SYS_PROMPT_1 = """ \
Context:
{context}
Based on the context provided above, answer the following question. Do not use any external knowledge.
If the answer is not present in the context, respond with "I don't know."
If the context is empty or irrelevant to the question, respond with "No relevant information found in the context."
"""

# okn: cites template did not really make a difference
SYS_PROMPT_2 = """ \
Context:
{context}
Based on the context provided above, answer the following question. Do not use any external knowledge.
Ensure your response cites the relevant context information for each fact you provide."
"""

# pt.SYSTEM_TEMPLATE = SYS_PROMPT_1

In [13]:
context_prompt = ""
for (score, text) in context:
    context_prompt += (text + "\\n")
print(context_prompt)

p_sys_ddic = pt.system_prompt(context_prompt)

 Mu, S. Kadavath, F. Wang, E. Dorundo, R. Desai, T. Zhu,
S. Parajuli, M. Guo, et al. The many faces of robustness: A critical analysis of out-of-
distribution generalization. In Proceedings of the IEEE/CVF International Conference on
Computer Vision, pages 8340–8349, 2021.
12
[35] D. Hendrycks, K. Zhao, S. Basart, J. Steinhardt, and D. Song. Natural adversarial examples.
In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition,
pages 15262–15271, 2021.
[36] S. Hooker, A. Courville, G. Clark, Y. Dauphin, and A. Frome. What do compressed deep
neural networks forget? arXiv preprint arXiv:1911.05248, 2019.
[37] S. Hooker, N. Moorosi, G. Clark, S. Bengio, and E. Denton. Characterising bias in compressed
models. arXiv preprint arXiv:2010.03058, 2020.
[38] H. Hotelling\nLab/robustness.
[25] A. Gholami, S. Kim, Z. Dong, Z. Yao, M. W. Mahoney, and K. Keutzer. A survey of
quantization methods for efficient neural network inference. arXiv preprint arXiv:2103.13630,
202

In [14]:
# view the full sy prompt comntent
print(p_sys_ddic['content'])

 Context information is below.
---------------------
 Mu, S. Kadavath, F. Wang, E. Dorundo, R. Desai, T. Zhu,
S. Parajuli, M. Guo, et al. The many faces of robustness: A critical analysis of out-of-
distribution generalization. In Proceedings of the IEEE/CVF International Conference on
Computer Vision, pages 8340–8349, 2021.
12
[35] D. Hendrycks, K. Zhao, S. Basart, J. Steinhardt, and D. Song. Natural adversarial examples.
In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition,
pages 15262–15271, 2021.
[36] S. Hooker, A. Courville, G. Clark, Y. Dauphin, and A. Frome. What do compressed deep
neural networks forget? arXiv preprint arXiv:1911.05248, 2019.
[37] S. Hooker, N. Moorosi, G. Clark, S. Bengio, and E. Denton. Characterising bias in compressed
models. arXiv preprint arXiv:2010.03058, 2020.
[38] H. Hotelling\nLab/robustness.
[25] A. Gholami, S. Kim, Z. Dong, Z. Yao, M. W. Mahoney, and K. Keutzer. A survey of
quantization methods for efficient neural ne

In [15]:
p_user_ddic = pt.user_prompt(user_query)

In [16]:
# view the full sy prompt comntent
print(p_user_ddic['content'])

 Query: 
Are there computational and statistical constraints?




In [17]:
# create list of system and user prompt dictionary
list_of_prompts = [
    pt.system_prompt(context_prompt),
    # pt.user_prompt(user_query + "\nProvide the context used for your explanations.")
    pt.user_prompt(user_query)
]

print(list_of_prompts)

[{'role': 'system', 'content': ' Context information is below.\n---------------------\n Mu, S. Kadavath, F. Wang, E. Dorundo, R. Desai, T. Zhu,\nS. Parajuli, M. Guo, et al. The many faces of robustness: A critical analysis of out-of-\ndistribution generalization. In Proceedings of the IEEE/CVF International Conference on\nComputer Vision, pages 8340–8349, 2021.\n12\n[35] D. Hendrycks, K. Zhao, S. Basart, J. Steinhardt, and D. Song. Natural adversarial examples.\nIn Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition,\npages 15262–15271, 2021.\n[36] S. Hooker, A. Courville, G. Clark, Y. Dauphin, and A. Frome. What do compressed deep\nneural networks forget? arXiv preprint arXiv:1911.05248, 2019.\n[37] S. Hooker, N. Moorosi, G. Clark, S. Bengio, and E. Denton. Characterising bias in compressed\nmodels. arXiv preprint arXiv:2010.03058, 2020.\n[38] H. Hotelling\\nLab/robustness.\n[25] A. Gholami, S. Kim, Z. Dong, Z. Yao, M. W. Mahoney, and K. Keutzer. A survey

# Generation

Let us generate a response using the augmented prompt.

First, initialize the generation model:

In [21]:
from py4ragTools.generation_tools import GenerationTools
#gt = GenerationTools(generation_model='llama3.1:8b')
gt = GenerationTools(generation_model='llama3.1:8b')

constructor here


In [22]:
response = gt.get_response(list_of_prompts) 
response.choices[0].message.content

"Yes, there are several computational and statistical constraints that can impact the reliability and validity of a study's findings. Here are some examples:\n\n**Computational Constraints:**\n\n1. **Computational power:** As data sizes increase, computational power requirements also grow exponentially. This can make it difficult to perform complex analyses or simulations on large datasets.\n2. **Algorithmic limitations:** The choice of algorithm or programming language can introduce limitations in terms of scalability, accuracy, or interpretability.\n3. **Data storage and preprocessing:** Managing and preprocessing large datasets can be computationally expensive and require significant resources.\n\n**Statistical Constraints:**\n\n1. **Sampling bias:** Sampling errors or biases can occur due to inadequate sampling techniques, which can limit the generalizability of findings.\n2. **Type I and Type II errors:** Failing to reject a false null hypothesis (Type I error) or accepting a true

In [23]:
print(response.choices[0].message.content)

Yes, there are several computational and statistical constraints that can impact the reliability and validity of a study's findings. Here are some examples:

**Computational Constraints:**

1. **Computational power:** As data sizes increase, computational power requirements also grow exponentially. This can make it difficult to perform complex analyses or simulations on large datasets.
2. **Algorithmic limitations:** The choice of algorithm or programming language can introduce limitations in terms of scalability, accuracy, or interpretability.
3. **Data storage and preprocessing:** Managing and preprocessing large datasets can be computationally expensive and require significant resources.

**Statistical Constraints:**

1. **Sampling bias:** Sampling errors or biases can occur due to inadequate sampling techniques, which can limit the generalizability of findings.
2. **Type I and Type II errors:** Failing to reject a false null hypothesis (Type I error) or accepting a true alternative